<a href="https://colab.research.google.com/github/shibakyrc0123-arch/A-Gps-Versiones/blob/main/Ver_1_2_%7C_A_Gps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd
from shapely.geometry import Point, Polygon
from geopy.geocoders import Nominatim
import time

# --- CONFIGURACIÓN ---
geolocator = Nominatim(user_agent="debug_tracker_v5")
bodega_coords = [
    (4.101740989185771, -73.65614987790829),
    (4.10275779142336, -73.65779357282884),
    (4.103576674506101, -73.65722519691684),
    (4.103303969059117, -73.65525092470286)
]
bodega_polygon = Polygon(bodega_coords)

def encontrar_archivo():
    # Busca el archivo automáticamente
    rutas = ['/content/', '/content/sample_data/']
    for r in rutas:
        if os.path.exists(r):
            for f in os.listdir(r):
                if (f.endswith('.xls') or f.endswith('.xlsx')) and "california" not in f:
                    return os.path.join(r, f)
    return None

def diagnostico():
    archivo = encontrar_archivo()
    if not archivo:
        print("❌ ERROR: No encuentro el archivo. ¿Lo subiste?")
        return

    print(f"📂 Leyendo archivo: {archivo}")
    try:
        df = pd.read_excel(archivo)
    except:
        try:
            df = pd.read_csv(archivo)
        except:
            print("❌ No pude leer el archivo como Excel ni CSV.")
            return

    # Normalizar columnas a MAYUSCULAS
    df.columns = df.columns.str.upper().str.strip()

    print(f"✅ Columnas encontradas: {list(df.columns)}")

    if 'EVENTO' not in df.columns:
        print("❌ NO VEO LA COLUMNA 'EVENTO'. Revisa los nombres arriba.")
        return

    # MUESTRA LOS EVENTOS ÚNICOS (Esto es la clave)
    print("\n--- 🔍 LISTA DE EVENTOS QUE CONTIENE TU ARCHIVO ---")
    eventos_unicos = df['EVENTO'].unique()
    for e in eventos_unicos:
        print(f" > '{e}'")
    print("---------------------------------------------------\n")

    # INTENTO DE PROCESAMIENTO CON PRINTS DE DEPURACIÓN
    print("--- INICIANDO SIMULACIÓN DE LÓGICA ---")

    contador_reporte = 0
    df['EVENTO'] = df['EVENTO'].astype(str).str.upper() # Todo a mayúsculas para comparar mejor

    # Variables estado
    dentro_bodega = False

    for i, row in df.iterrows():
        ev = row['EVENTO']

        # PRUEBA DE BODEGA
        try:
            coords = str(row['COORDS']).replace(" ", "").split(',')
            punto = Point(float(coords[0]), float(coords[1]))
            en_bodega = bodega_polygon.contains(punto)

            if en_bodega and not dentro_bodega:
                print(f"📍 DETECTADO ENTRADA BODEGA en fila {i}")
                dentro_bodega = True
                contador_reporte += 1
            elif not en_bodega and dentro_bodega:
                print(f"📍 DETECTADO SALIDA BODEGA en fila {i}")
                dentro_bodega = False
                contador_reporte += 1
        except:
            pass # Error coordenadas

        # PRUEBA DE TEXTO (BUSQUEDA FLEXIBLE)
        if "ENCENDIDO" in ev:
            print(f"⚡ Encontré ENCENDIDO en fila {i}: '{ev}'")
            contador_reporte += 1

        if "ESTACIONADO" in ev:
            # Solo imprimimos el primero para no llenar la pantalla
            if contador_reporte < 5:
                print(f"🅿️ Encontré ESTACIONADO en fila {i}: '{ev}'")
            contador_reporte += 1

        if "ENTRADA" in ev or "SALIDA" in ev:
             print(f"🏙️ Encontré CIUDAD en fila {i}: '{ev}'")
             contador_reporte += 1

    print("\n---------------------------------------------------")
    if contador_reporte == 0:
        print("❌ RESULTADO: 0 eventos detectados.")
        print("⚠️ EL PROBLEMA: El código busca palabras clave como 'ENCENDIDO', 'ESTACIONADO', 'ENTRADA'.")
        print("Mira la lista de eventos arriba y dime cómo están escritos exactamente.")
    else:
        print(f"✅ ÉXITO: Se detectaron {contador_reporte} eventos potenciales.")
        print("Si ves esto, el código anterior falló por algo sutil. ¿Quieres que genere el reporte ahora?")

diagnostico()

📂 Leyendo archivo: /content/sample_data/reporte-NCX43H-15-01-2026-10-06-12.xlsx
✅ Columnas encontradas: ['EVENTO', 'FHEVENTO', 'VEL', 'COORDS', 'ODOM', 'UNNAMED: 5']

--- 🔍 LISTA DE EVENTOS QUE CONTIENE TU ARCHIVO ---
 > 'Vehiculo Estacionado'
 > 'Vehiculo Encendido'
 > 'Reporte Por Tiempo'
 > 'Vehiculo en Movimiento'
 > 'Frenada Brusca'
 > 'Aceleracion Brusca'
 > 'Vehiculo Apagado'
 > 'Giro Brusco'
---------------------------------------------------

--- INICIANDO SIMULACIÓN DE LÓGICA ---
🅿️ Encontré ESTACIONADO en fila 0: 'VEHICULO ESTACIONADO'
🅿️ Encontré ESTACIONADO en fila 1: 'VEHICULO ESTACIONADO'
🅿️ Encontré ESTACIONADO en fila 2: 'VEHICULO ESTACIONADO'
🅿️ Encontré ESTACIONADO en fila 3: 'VEHICULO ESTACIONADO'
🅿️ Encontré ESTACIONADO en fila 4: 'VEHICULO ESTACIONADO'
⚡ Encontré ENCENDIDO en fila 10: 'VEHICULO ENCENDIDO'
⚡ Encontré ENCENDIDO en fila 22: 'VEHICULO ENCENDIDO'
⚡ Encontré ENCENDIDO en fila 30: 'VEHICULO ENCENDIDO'
⚡ Encontré ENCENDIDO en fila 40: 'VEHICULO ENCENDIDO'